<h1>Distributed Hyperparameter Optimization (HPO) Techniques for CNN on MNIST</h1>

In [2]:
# TO BE DELETED ONCE COMPLETE
%pip install torchvision
%pip install optuna
%pip install hpbandster
%pip install ConfigSpace
%pip install torch
%pip install torchsummary
%pip install plotly
%pip install matplotlib


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1


<h2>1. Introduction</h2>

Hyperparameter Optimization (HPO) is a critical step in deep learning model training to improve accuracy and efficiency. 
Traditional hyperparameter tuning approaches like Grid Search and Random Search are computationally expensive and inefficient. 

In this assignment, we compare and analyze different hyperparameter optimization strategies using distributed computing to achieve optimal hyperparameter selection efficiently.

<h2>2. Objectives</h2>

The goal of this project is to:

1. Compare multiple HPO techniques for training a Convolutional Neural Network (CNN) on the MNIST dataset.

2. Evaluate these techniques based on training speed, search efficiency, accuracy, and GPU resource utilization.

3. Implement real-time GPU monitoring to track memory usage and optimize resource allocation.

4. Identify the most effective HPO method that balances speed, accuracy, and efficiency.

<h2>3. HPO Strategies Implemented</h2>

We implemented and compared four different approaches for HPO:

1. Baseline (No HPO): Train the model with default hyperparameters.

2. ASHA (Asynchronous Successive Halving Algorithm): Eliminates underperforming trials early to speed up training.

3. BOHB (Bayesian Optimization + HyperBand): Uses Bayesian learning to intelligently select hyperparameters while efficiently allocating compute resources.

4. BOHB + ASHA Hybrid: Combines BOHB’s smart selection with ASHA’s aggressive pruning for improved efficiency.

<h2>4. Implementation Details</h2>

<h2>4.1 Dataset: MNIST</h2>

The MNIST dataset consists of handwritten digits (0-9).

Training set: 1000 images.

Test set: 1000 images.

Image size: 28x28 pixels, grayscale.

Output classes: 10 (digits 0-9).

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset

import optuna
from optuna.pruners import SuccessiveHalvingPruner

import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres
from hpbandster.optimizers.bohb import BOHB
from hpbandster.core.worker import Worker
import ConfigSpace as CS

from torch.utils.tensorboard import SummaryWriter
import ssl
import time
import psutil
import random

ssl._create_default_https_context = ssl._create_unverified_context

2025-03-18 12:05:33.722277: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:

# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load full MNIST dataset
full_trainset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
full_testset = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# Select 1000 random indices for train and test sets
train_indices = np.random.choice(len(full_trainset), 10000, replace=False)
test_indices = np.random.choice(len(full_testset), 10000, replace=False)

# Create subsets of MNIST
trainset = Subset(full_trainset, train_indices)
testset = Subset(full_testset, test_indices)

# Create DataLoaders
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

dataset = (trainloader, testloader)


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:02<00:00, 4484191.57it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 143461.96it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:01<00:00, 1479402.78it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 4570664.29it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



<h2>4.2 Model: CNN Architecture</h2>

The CNN model used for training consists of:

1. Two convolutional layers with ReLU activation.

2. Max-pooling layers for feature down-sampling.

3. Fully connected layers with a dropout layer.

4. Softmax activation for classification.

<b>Hyperparameters Considered</b>

Learning Rate - 1e-4 to 1e-2 (log scale)

Dropout Rate - 0.2 to 0.5

Number of Filters - 16, 32, 64


In [5]:


# CNN Model for MNIST
class CNN(nn.Module):
    def __init__(self, dropout_rate=0.5, num_filters=32):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, num_filters, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(num_filters, num_filters * 2, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(num_filters * 2 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

<h2>4.3 GPU Monitoring & Resource Utilization Tracking</h2>

We implemented real-time GPU monitoring using PyTorch’s memory allocation tracking.

GPU usage was recorded at each training epoch.

This allowed us to compare memory efficiency across different HPO techniques.

In [6]:
# Function to log memory usage (CPU + GPU approximation)
def log_memory_usage(stage=""):
    # Get CPU RAM usage
    ram_usage = psutil.virtual_memory().used / (1024 ** 2)  # Convert to MB
    
    # Get GPU memory (Approximate via tensor usage)
    if device.type == "mps":
        torch.mps.empty_cache()  # Free unused memory (for better tracking)
        gpu_usage = "MPS does not expose memory tracking"
    else:
        gpu_usage = "GPU not in use"
    
    return ram_usage, gpu_usage

<h2>5. Comparison of HPO Approaches</h2>

1. Training Speed
2. Model Accuracy
3. GPU Memory Utilization

<h2>5.1 Baseline Model (No HPO)</h2>

In [7]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [8]:


# Train Baseline Model (Without HPO) with GPU Logging
def train_baseline():
    writer = SummaryWriter(log_dir="./logs/baseline")
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    print(f"Using device: {device}")
    model = CNN().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.CrossEntropyLoss()
    
    start_time = time.time()
    # gpu_usages = []
    memory_logs = []

    for epoch in range(5):
        model.train()
        epoch_loss = 0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        # Log GPU Memory
        # gpu_usage = log_gpu_usage("Baseline")
        # gpu_usages.append(gpu_usage)
        
        # Log Memory Usage
        ram_usage, gpu_usage = log_memory_usage("Baseline")
        memory_logs.append(ram_usage)
        
        writer.add_scalar("Loss/train", epoch_loss / len(trainloader), epoch)
        writer.add_scalar("Memory/CPU_RAM_MB", ram_usage, epoch)
    
    end_time = time.time()
    
    # Compute GPU Usage Stats
    # avg_gpu_usage = sum(gpu_usages) / len(gpu_usages)
    # avg_gpu_usage = "MPS memory tracking unavailable"

    # Compute Average Memory Usage
    avg_ram_usage = sum(memory_logs) / len(memory_logs)

    # Test Model
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Baseline Accuracy: {accuracy:.2f}%, Training Time: {end_time - start_time:.2f}s, Avg CPU RAM Usage: {avg_ram_usage:.2f} MB, GPU Usage: {gpu_usage}")
    writer.close()

    # return accuracy, end_time - start_time, avg_gpu_usage
    return accuracy, end_time - start_time, avg_ram_usage
    

# Run Baseline Training
# baseline_accuracy, baseline_time, baseline_gpu = train_baseline()
baseline_accuracy, baseline_time, baseline_memory = train_baseline()


Using device: mps
Baseline Accuracy: 98.24%, Training Time: 38.88s, Avg CPU RAM Usage: 8611.47 MB, GPU Usage: MPS does not expose memory tracking


<h2>5.2 ASHA HPO</h2>

In [11]:


# Train Model with ASHA HPO, Memory Logging, and TensorBoard Logging
def train_cnn_asha(trial):
    writer = SummaryWriter(log_dir=f"./logs/asha_trial_{trial.number}")  # TensorBoard log directory
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

    # Sample hyperparameters using Optuna
    dropout_rate = trial.suggest_float("dropout", 0.2, 0.5)
    num_filters = trial.suggest_categorical("num_filters", [16, 32, 64])
    learning_rate = trial.suggest_float("lr", 1e-4, 1e-2, log=True)

    model = CNN(dropout_rate=dropout_rate, num_filters=num_filters).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()

    memory_logs = []
    start_time = time.time()

    for epoch in range(5):
        model.train()
        epoch_loss = 0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        # Log Memory Usage
        ram_usage, gpu_usage = log_memory_usage("ASHA")
        memory_logs.append(ram_usage)

        # Log Loss and Memory to TensorBoard
        writer.add_scalar("Loss/train", epoch_loss / len(trainloader), epoch)
        writer.add_scalar("Memory/CPU_RAM_MB", ram_usage, epoch)

        # Evaluate Model (ASHA needs validation accuracy for pruning)
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total

        # Report accuracy for ASHA pruning
        trial.report(accuracy, epoch)

        # ASHA: Stop bad trials early
        if trial.should_prune():
            writer.close()  # Ensure writer closes even when pruned
            raise optuna.exceptions.TrialPruned()

    end_time = time.time()

    # Compute Average CPU Memory Usage
    avg_ram_usage = sum(memory_logs) / len(memory_logs)

    # Log final accuracy and memory stats to TensorBoard
    writer.add_scalar("Accuracy", accuracy)
    writer.add_scalar("Training Time (s)", end_time - start_time)
    writer.close()

    # Print Summary (Same Format as Baseline)
    print(f"Accuracy: {accuracy:.2f}%, Training Time: {end_time - start_time:.2f}s, "
          f"Avg CPU RAM Usage: {avg_ram_usage:.2f} MB, GPU Usage: {gpu_usage}")

    return accuracy, end_time - start_time, avg_ram_usage, gpu_usage

In [18]:
import optuna
from optuna.pruners import SuccessiveHalvingPruner
import multiprocessing

# Store best training time & resource usage
best_training_time = float("inf")
best_ram_usage = float("inf")
best_gpu_usage = None

# Optimize parallel processing
# n_jobs = max(1, multiprocessing.cpu_count() // 2)  # Use half the available cores

# Enable best GPU performance for Apple MPS
torch.set_float32_matmul_precision('high') 

# Define Objective Function for Optuna
def objective(trial):
    global best_training_time, best_ram_usage, best_gpu_usage

    accuracy, training_time, avg_ram_usage, gpu_usage = train_cnn_asha(trial)  # Now returns more metrics

    # Track best training time & resource utilization
    if training_time < best_training_time:
        best_training_time = training_time
        best_ram_usage = avg_ram_usage
        best_gpu_usage = gpu_usage

    return accuracy  # Optuna optimizes based on accuracy

# Create Optuna Study with ASHA (Successive Halving)
study = optuna.create_study(
    study_name="asha_hpo",
    direction="maximize",  # We want to maximize accuracy
    pruner=SuccessiveHalvingPruner(),  # ASHA Pruning
    sampler=optuna.samplers.TPESampler(
        multivariate=True,  # Optimizes multiple parameters together
        constant_liar=True  # Avoids redundant evaluations
    )
)

# Run Optimization (20 Trials)
study.optimize(objective, n_trials=10)

# Print Best Results
print(f"\nBest Model Config: {study.best_params}")
print(f"Best Accuracy: {study.best_value:.2f}%")
print(f"Best Training Time: {best_training_time:.2f}s")
print(f"Best Avg CPU RAM Usage: {best_ram_usage:.2f} MB")
print(f"Best GPU Usage: {best_gpu_usage}")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``constant_liar`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-03-18 12:15:15,782] A new study created in memory with name: asha_hpo
[I 2025-03-18 12:16:16,141] Trial 0 finished with value: 97.66 and parameters: {'dropout': 0.39120732202654807, 'num_filters': 64, 'lr': 0.00029045640929732837}. Best is trial 0 with value: 97.66.


Accuracy: 97.66%, Training Time: 59.36s, Avg CPU RAM Usage: 8869.64 MB, GPU Usage: MPS does not expose memory tracking


[I 2025-03-18 12:17:10,045] Trial 1 finished with value: 98.23 and parameters: {'dropout': 0.4177646823620649, 'num_filters': 64, 'lr': 0.0010718539614060052}. Best is trial 1 with value: 98.23.


Accuracy: 98.23%, Training Time: 53.88s, Avg CPU RAM Usage: 8396.55 MB, GPU Usage: MPS does not expose memory tracking


[I 2025-03-18 12:18:01,179] Trial 2 pruned. 
[I 2025-03-18 12:18:24,295] Trial 3 pruned. 
[I 2025-03-18 12:18:52,496] Trial 4 pruned. 
[I 2025-03-18 12:19:20,421] Trial 5 pruned. 
[I 2025-03-18 12:19:40,054] Trial 6 pruned. 
[I 2025-03-18 12:20:00,200] Trial 7 pruned. 
[I 2025-03-18 12:20:20,191] Trial 8 pruned. 
[I 2025-03-18 12:20:39,689] Trial 9 pruned. 



Best Model Config: {'dropout': 0.4177646823620649, 'num_filters': 64, 'lr': 0.0010718539614060052}
Best Accuracy: 98.23%
Best Training Time: 53.88s
Best Avg CPU RAM Usage: 8396.55 MB
Best GPU Usage: MPS does not expose memory tracking


In [25]:
%pip install plotly


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from optuna.visualization import plot_optimization_history, plot_param_importances


# Plot optimization history
optimization_history_fig = plot_optimization_history(study)
optimization_history_fig.show()

# Plot parameter importances
param_importances_fig = plot_param_importances(study)
param_importances_fig.show()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'study' is not defined

<h2>5.3 Train with BOHB HPO</h2>

In [27]:


# Define Worker for BOHB
class CNNWorker(Worker):
    def __init__(self, run_id, dataset, **kwargs):
        # print('__init__')
        super().__init__(run_id, **kwargs)
        self.trainloader, self.testloader = dataset
        self.device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

    def compute(self, config, budget, **kwargs):
        print(type(config))
        print(type(budget))
        writer = SummaryWriter(log_dir=f"./logs/bohb_trial_{config}")
        print('writer')

        # ✅ Convert `config` to Python native dict
        config_native = {key: int(value) if isinstance(value, np.integer) else float(value) if isinstance(value, np.floating) else value for key, value in config.items()}
        
        model = CNN(dropout_rate=float(config["dropout"]), num_filters=int(config["num_filters"])).to(self.device)
        optimizer = optim.Adam(model.parameters(), lr=float(config["lr"]))
        loss_fn = nn.CrossEntropyLoss()
        print('reached loss_fn')

        memory_logs = []
        start_time = time.time()

        for epoch in range(int(budget)):  # `budget` is set by BOHB (early stopping)
            model.train()
            epoch_loss = 0
            for images, labels in self.trainloader:
                images, labels = images.to(self.device), labels.to(self.device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = loss_fn(outputs, labels)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()

            # Log Memory Usage
            ram_usage, gpu_usage = log_memory_usage()
            memory_logs.append(float(ram_usage))

            # Log Loss and Memory to TensorBoard
            writer.add_scalar("Loss/train", epoch_loss / len(self.trainloader), epoch)
            writer.add_scalar("Memory/CPU_RAM_MB", ram_usage, epoch)

        end_time = time.time()
        avg_ram_usage = sum(memory_logs) / len(memory_logs)

        # Evaluate Model
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in self.testloader:
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total

        # ✅ Convert all NumPy types to standard Python types before returning
        accuracy = int(np.round(accuracy))  # Convert NumPy int64 to Python int
        avg_ram_usage = float(np.round(avg_ram_usage, 2))  # Convert float32 to Python float
        training_time = float(np.round(end_time - start_time, 2))  # Convert time to Python float

        writer.add_scalar("Accuracy", accuracy)
        writer.add_scalar("Training Time (s)", training_time)
        writer.close()

        print(f"Accuracy: {accuracy:.2f}%, Training Time: {training_time:.2f}s, "
              f"Avg CPU RAM Usage: {avg_ram_usage:.2f} MB, GPU Usage: {gpu_usage}")

        return {
        "loss": -accuracy,  # Loss should be negative for BOHB to maximize accuracy
        "info": {
            "training_time": training_time,
            "ram_usage": avg_ram_usage,
            "config": config_native  # ✅ Ensure all values are JSON serializable
            }
        }

    @staticmethod
    def get_configspace():
        cs = CS.ConfigurationSpace()
        cs.add(CS.UniformFloatHyperparameter("dropout", lower= float(0.2), upper= float(0.5)))
        cs.add(CS.CategoricalHyperparameter("num_filters", choices=[16, 32, 64]))
        cs.add(CS.UniformFloatHyperparameter("lr", lower=float(0.0001), upper=float(0.01)))
        # print('CS')
        return cs

In [28]:
def get_configspace():
    config_space = {
        "dropout": {"type": "float", "lower": 0.2, "upper": 0.5},
        "num_filters": {"type": "categorical", "choices": [16, 32, 64]},
        "lr": {"type": "float", "lower": 0.0001, "upper": 0.01}
    }
    return config_space


In [29]:


def sample_hyperparameters(config_space):
    sampled_config = {}
    for param, details in config_space.items():
        if details["type"] == "float":
            sampled_config[param] = random.uniform(details["lower"], details["upper"])
        elif details["type"] == "categorical":
            sampled_config[param] = random.choice(details["choices"])
    return sampled_config


In [30]:
config_space = get_configspace()
sampled_config = sample_hyperparameters(config_space)
print("Sampled Hyperparameters:", sampled_config)


Sampled Hyperparameters: {'dropout': 0.2907754557882343, 'num_filters': 64, 'lr': 0.001997670302093464}


In [31]:
# Set up BOHB optimization
run_id = "bohb_hpo"
# result_logger = hpres.json_result_logger(directory="./bohb_results", overwrite=True)

# Start Nameserver for BOHB
NS = hpns.NameServer(run_id=run_id, host="localhost", port=0)
NS.start()

# Start BOHB Worker
worker = CNNWorker(run_id=run_id, dataset=dataset, nameserver="localhost", nameserver_port=NS.port)
worker.run(background=True)

print(CNNWorker.get_configspace())

# Run BOHB Optimization
bohb = BOHB(
    configspace=sampled_config,
    run_id=run_id,
    nameserver="localhost",
    nameserver_port=NS.port,
    min_budget= int(1),  # Minimum epochs per trial
    max_budget= int(5)  # Maximum epochs per trial
    # result_logger=result_logger
)

res = bohb.run(n_iterations= int(2))  # Number of trials

# # Shutdown Nameserver and Worker
# bohb.shutdown(shutdown_workers=True)
# NS.shutdown()

# # Get Best Hyperparameters
# best_config = res.get_incumbent_id()
# best_accuracy = -res.get_incumbent_trajectory()["losses"][-1]

# print(f"\n🔹 Best Model Config: {res.get_id2config_mapping()[best_config]['config']}")
# print(f"✅ Best Accuracy: {best_accuracy:.2f}%")

11:25:52 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x1a847eb50; connected IPv4; for PYRO:Pyro.NameServer@localhost:52997>
11:25:52 WORKER: No dispatcher found. Waiting for one to initiate contact.
11:25:52 WORKER: start listening for jobs


Configuration space object:
  Hyperparameters:
    dropout, Type: UniformFloat, Range: [0.2, 0.5], Default: 0.35
    lr, Type: UniformFloat, Range: [0.0001, 0.01], Default: 0.00505
    num_filters, Type: Categorical, Choices: {16, 32, 64}, Default: 16



AttributeError: 'dict' object has no attribute 'get_hyperparameters'